In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import as_completed

In [7]:
def html_select(html, path):
    return html.select(path)

In [10]:
years = [str(year + 2000) for year in range(22)]

id_list = '#Myform > div.ss_book_box'
title_path = 'a.Ere_bo_title'
price_path = 'span.Ere_fs24'
page_path = 'div.conts_info_list1 > ul > li'
type_path = 'div.conts_info_list2 > ul > li > a'
bestsellers = []
for year in years:
    BASE_URL = f"https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=YearlyBest&BranchType=1&CID=0&Year={year}"
    original_html = requests.get(BASE_URL)
    html = BeautifulSoup(original_html.text, "html.parser")
    books = html_select(html, id_list)
    book_id_list = [item["itemid"] for item in books]
    
    
    for index, itemid in enumerate(book_id_list) : 
        product_url = f"https://www.aladin.co.kr/shop/wproduct.aspx?ItemId={itemid}"
        product_html = requests.get(product_url)
        html = BeautifulSoup(product_html.text, "html.parser")
        
        book_title = html.select(title_path)[0].get_text()
        book_price = html.select(price_path)[0].get_text()
        book_page = html.select(page_path)[0].get_text()
        
        if book_page == "양장본":
            book_page = html.select(page_path)[1].get_text()
                
        book_type = html.select(type_path)[1].get_text()
        
        bestsellers.append([year, index+1, book_title, book_price, book_page, book_type])

In [12]:
df = pd.DataFrame(bestsellers, columns=["연도","순위","제목", "가격","페이지","장르"])
df.to_excel('./bestsellers.xlsx')